<a href="https://colab.research.google.com/github/RobertMannix/Data_Science/blob/main/Robert_Mannix_Stats_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STA 2023 Final Project
## Directions:

1) Replace the image dataset in the google slides with your own data.

2) Comment and explain all code to the best of your ability.

3) Run your model 3 times. 

4) Report best, worst, and average of the three runs. 




Use any and all resources available to you.

Comment and explain the code.

Answer the questions to the best of your abilities.

If you have any questions, txt 561-479-8123

Save this notebook to your GitHub and submit the link on Canvas


In [1]:
%%capture          # imports that allow for us to have access to these functions
!pip install wandb
!apt-get install poppler-utils
!pip install pdf2image
!pip install flashtorch
import requests
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
import numpy as np
import torch
import requests
from torchvision import *
from torchvision.models import *
from flashtorch.utils import apply_transforms
import wandb as wb

UsageError: unrecognized arguments: imports that allow for us to have access to these functions


In [ ]:
def GPU(data):     
    return torch.tensor(data, requires_grad=True, dtype=torch.float, device=torch.device('cuda'))
                      
def GPU_data(data):
    return torch.tensor(data, requires_grad=False, dtype=torch.float, device=torch.device('cuda'))

def plot(x): # line of best fit set up
    fig, ax = plt.subplots()
    im = ax.imshow(x, cmap = 'gray')
    ax.axis('off')
    fig.set_size_inches(5, 5)
    plt.show()
    
def get_google_slide(url): # colab grabbing my images from the google slides to put on line of best fit
    url_head = "https://docs.google.com/presentation/d/"
    url_body = url.split('/')[5]
    page_id = url.split('.')[-1]
    return url_head + url_body + "/export/pdf?id=" + url_body + "&pageid=" + page_id

def get_slides(url): 
    url = get_google_slide(url)
    r = requests.get(url, allow_redirects=True)
    open('file.pdf', 'wb').write(r.content)
    images = convert_from_path('file.pdf', 500)
    return images

def load(image):

    return apply_transforms(image).clone().detach().requires_grad_(True).to(device)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
labels = {int(key):value for (key, value) in requests.get('https://s3.amazonaws.com/mlpipes/pytorch-quick-start/labels.json').json().items()}

model = alexnet(weights='DEFAULT').to(device) # all the options of things the ai will recognize; like a word bank of things it knows
model.eval();

In [ ]:
url = "https://docs.google.com/presentation/d/11UEqO-6oLyiMv2F1iCh1Q9AisKjdL58QtoPreax_qqs/edit#slide=id.g1a1a0885e5a_0_197" # my images url

In [ ]:
images = [] # displays the images in my slides

for image in get_slides(url):

    plot(image)

    images.append(load(image))

images = torch.vstack(images)

In [ ]:
images.shape # gives the size of the images

In [ ]:
model(images) 

In [ ]:
y = model(images) 

In [ ]:
y.shape 

In [ ]:
guesses = torch.argmax(y, 1).cpu().numpy()

In [ ]:
for i in list(guesses):  # each thing is what the ai recogizes my images to be based on the things it knows are available
    print(labels[i])

In [ ]:
Y = np.zeros(50,) 
Y[25:] = 1

In [ ]:
Y # zeros are the first 25 and 1 is the second 25 images. Telling the computer it is different images

In [ ]:
X = y.detach().cpu().numpy() 

In [ ]:
X.shape

In [ ]:
plt.plot(X[0],'.') # a line of best fit for the ai based on my images and how accurate it is

In [ ]:
X[0] #all the images I have in pixel form

In [ ]:
np.argmax(X[0]) # Argmax returns the indices of the maximum values along an axis

In [ ]:
labels[948]

In [ ]:
top_ten = np.argsort(X[0])[::-1][0:10] # used to perform an indirect sort along the given axis using the algorithm specified by the kind keyword

In [ ]:
for i in top_ten: # top ten most common resposes for what the ai thought my images were
    print(labels[i])

In [ ]:
labels # every thing the computer knows and could give as a response to my images

In [ ]:
plt.hist(X[0]) # histogram showing the responses of what the ai thought were my images and how accurate they were

In [ ]:
X = GPU_data(X)
Y = GPU_data(Y)

In [ ]:
def softmax(x): # converts a vector of numbers into a vector of probabilities
    s1 = torch.exp(x - torch.max(x,1)[0][:,None])
    s = s1 / s1.sum(1)[:,None]
    return s

In [ ]:
def cross_entropy(outputs, labels):      # a measure of the difference between two probability distributions for a given random variable or set of events      
    return -torch.sum(softmax(outputs).log()[range(outputs.size()[0]), labels.long()])/outputs.size()[0]  

In [ ]:
def randn_trunc(s): #Truncated Normal Random Numbers
    mu = 0 
    sigma = 0.1
    R = stats.truncnorm((-2*sigma - mu) / sigma, (2*sigma - mu) / sigma, loc=mu, scale=sigma)
    return R.rvs(s)

In [ ]:
def Truncated_Normal(size):
    u1 = torch.rand(size)*(1-np.exp(-2)) + np.exp(-2)
    u2 = torch.rand(size)
    z  = torch.sqrt(-2*torch.log(u1)) * torch.cos(2*np.pi*u2)

    return z

In [ ]:
def acc(out,y):
    with torch.no_grad():
        return (torch.sum(torch.max(out,1)[1] == y).item())/y.shape[0]

In [ ]:
X.shape

In [ ]:
def get_batch(mode):
    b = c.b
    if mode == "train":
        r = np.random.randint(X.shape[0]-b) 
        x = X[r:r+b,:]
        y = Y[r:r+b]
    elif mode == "test":
        r = np.random.randint(X_test.shape[0]-b)
        x = X_test[r:r+b,:]
        y = Y_test[r:r+b]
    return x,y

In [ ]:
def model(x,w):

    return x@w[0]

In [ ]:
def make_plots(): #creating plots for the line of best fit
    
    acc_train = acc(model(x,w),y)
    
    # xt,yt = get_batch('test')

    # acc_test = acc(model(xt,w),yt)

    wb.log({"acc_train": acc_train})

In [ ]:
wb.init(project="Linear_Model_Photo_1");
c = wb.config

c.h = 0.001 # showing the slope and y intercept of the line of best fit
c.b = 32
c.epochs = 100000

w = [GPU(Truncated_Normal((1000,2)))]

optimizer = torch.optim.Adam(w, lr=c.h)  

for i in range(c.epochs):

    x,y = get_batch('train')

    loss = cross_entropy(softmax(model(x,w)),y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
   
    wb.log({"loss": loss})

    make_plots()


https://tensorspace.org/

Best, Worst, Average loss

Best - giddy-darkness-4 - 0.3261

Worst - warm-aardvark-5 - 0.3253

Average - 0.3256